In [3]:
%pip install openai
%pip install google.generativeai
import openai
import os
import google.generativeai as genai
import subprocess
import os
import pathlib
import textwrap
import subprocess
import os
import json
import re
import pathlib
import textwrap
# from  google.colab import drive
# from google.colab import userdata
from IPython.display import display
from IPython.display import Markdown
from openai import OpenAI

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
train = pd.read_csv('df_train_shuffled.csv' )

In [5]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
#path to your folder which contains the json and other files
with open('D:\LLM_GPT\APPCAIR\expDetails.json','r') as openfile:
  expDetails = json.load(openfile)

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

GOOGLE_API_KEY=expDetails['GeminiAPIKEY']
#CLAUDE_API_KEY= expDetails['ClaudeAPIKEY']
GPT_API_KEY = expDetails['GPTAPIKEY']
os.environ['OPENAI_API_KEY'] = GPT_API_KEY

genai.configure(api_key=GOOGLE_API_KEY)

Gmodel = genai.GenerativeModel(expDetails['GeminiModel'])

GPTModel = expDetails['GPTModel']
openai.api_key = GPT_API_KEY
client = OpenAI()

RunCount = expDetails['RunCount']
RunCount = str(RunCount)
EXP_NAME = expDetails['ExpName']
json_object = json.dumps(expDetails,indent = 4)
with open('D:\LLM_GPT\APPCAIR\expDetails.json','w') as openfile:
  openfile.write(json_object)

def fileReader(pathToFile):
  f = open(pathToFile,'r')
  s = f.read()
  f.close()
  return s

idea = fileReader(expDetails['ideaFilePath'])
context = fileReader(expDetails['contextFilePath'])
libraries = fileReader(expDetails['librariesPath'])
methodology =fileReader(expDetails['methodology'])
miscellaneous= '' #fileReader(expDetails['miscellenous'])
idea = "\n\n"+ idea+"\n\n"+ methodology
custom_dataset=expDetails['dataset']
packages=fileReader(expDetails['packages'])

<>:1: SyntaxWarning: invalid escape sequence '\L'
<>:27: SyntaxWarning: invalid escape sequence '\L'
<>:1: SyntaxWarning: invalid escape sequence '\L'
<>:27: SyntaxWarning: invalid escape sequence '\L'
C:\Users\WIN 11\AppData\Local\Temp\ipykernel_25180\2832552133.py:1: SyntaxWarning: invalid escape sequence '\L'
  with open('D:\LLM_GPT\APPCAIR\expDetails.json','r') as openfile:
C:\Users\WIN 11\AppData\Local\Temp\ipykernel_25180\2832552133.py:27: SyntaxWarning: invalid escape sequence '\L'
  with open('D:\LLM_GPT\APPCAIR\expDetails.json','w') as openfile:


In [6]:
import subprocess
# Read the contents of the text file
packages = packages.replace('!', '')
# Execute pip install commands
subprocess.run(packages.split())

CompletedProcess(args=['pip', 'install', '-q', 'numerapi', 'umap-learn', 'pandas', 'pyarrow', 'matplotlib', 'lightgbm', 'scikit-learn', 'cloudpickle', 'scipy==1.10.1', 'tabpfn'], returncode=1)

In [7]:
SELF_CONSISTENCY = 5
chats = [None]*SELF_CONSISTENCY
responses = [None]*SELF_CONSISTENCY
for i in range(SELF_CONSISTENCY):
  chat  = Gmodel.start_chat(history=[])
  chats[i] = chat
InstructionModel = expDetails['InstructionGenerationModel']
InstructionModel = InstructionModel.lower()

In [8]:
InstructionModel='gpt' #if you want to use gpt4 for pipeline generation
modifiedPrelude = """
  You will be provided with an idea and the necessary libraries. Your task is to outline a detailed pipeline without giving actual code.

  Review the provided context and idea carefully.
  Do not include the code; focus on describing the pipeline.
  Explain each step thoroughly, considering the dataset characteristics and the proposed approach.
  Utilize the provided libraries as necessary.
  Aim for clarity and coherence in your response.

  Approach this as a Machine Learning Researcher, providing a step-by-step plan for the analysis.

  """

if InstructionModel == 'gemini':
    responses = [None] * SELF_CONSISTENCY
    for i in range(SELF_CONSISTENCY):
        try:
            responses[i] = Gmodel.generate_content(f"""
            {modifiedPrelude}
            [Idea]:{idea}
            """, safety_settings={'HARASSMENT': 'block_none'})
        except Exception as e:
            print(f"Error occurred with 'gemini' model: {e}")
            InstructionModel = 'gpt'
            break

if InstructionModel == 'gpt':
    responses = [None] * SELF_CONSISTENCY
    for i in range(SELF_CONSISTENCY):
        responses[i] = client.chat.completions.create(
            model=GPTModel,
            messages=[
                {"role": "system", "content": "You are a Machine Learning Researcher. You will be provided with an idea and the necessary libraries. Your task is to outline a detailed pipeline without giving actual code."},
                {"role": "user", "content": f"""
                {modifiedPrelude}
                [Idea]:{idea}
                """}
            ]
        )


In [12]:
import os
InstructionModel='gpt'
# Function to execute the script and return True if successful, False otherwise
def run_script():
  if(InstructionModel=='gemini'):
    try:
        # Define the self consistency prompt
        selfConsistencyPrompt = f"""I have generated the following responses to the following responses to the question:{modifiedPrelude}
          [Context]:{context}
          [Idea]:{idea}
           """
        # Append generated responses to the prompt
        for i in range(SELF_CONSISTENCY):
            selfConsistencyPrompt += f"""Response {i}:

          {responses[i].text}
          """

        # Append final prompt
        selfConsistencyPrompt += """
    Pick the most consistent response based on the majority consensus and give reason for choosing why that response
    """

        # Generate final response
        finalResponse = Gmodel.generate_content(selfConsistencyPrompt, safety_settings={'HARASSMENT':'block_none'})

        # Extract response number from the text
        Res = Gmodel.generate_content(f"Extract the response number from the text. Be very concise and output only and only the response number and nothing else {finalResponse.text}", safety_settings={'HARASSMENT':'block_none'})

        # Get instruction based on selected response
        Instruction = responses[int(Res.text)].text

        Instruction=""
        if(InstructionModel=="gpt"):
          # Get instruction based on selected response
          Instruction = responses[int(Res.choices[0].message.content)]
        else:
          Instruction = responses[int(Res.text)]
        print(Instruction)  
        Pipeline=Instruction.choices[0].message.content     
        base_folder = expDetails['FolderName'].lstrip('/').replace('/', '\\')  # Convert forward slashes to backslashes
        expname = expDetails['ExpName'] + RunCount
        # Write instruction to pipeline.md in the created folder
        filepath = os.path.join(base_folder, expname, 'pipeline.md')
        os.makedirs(os.path.dirname(filepath), exist_ok=True)
        with open(filepath,'w') as f:
            print("here4")
            f.write(Pipeline)

        return True  
    except Exception as e:
        print("Error occurred:", e)
        InstructionModel=='gpt'
          # Return False if there's an error

  if(InstructionModel=='gpt'):
      try:
        # Define the self consistency prompt
        selfConsistencyPrompt = f"""I have generated the following responses to the following responses to the question:{modifiedPrelude}
          [Context]:{context}
          [Idea]:{idea}
           """

        # Append generated responses to the prompt
        for i in range(SELF_CONSISTENCY):
            selfConsistencyPrompt += f"""Response {i}:

          {responses[i]}
          """

        # Append final prompt
        selfConsistencyPrompt += """
    Pick the most consistent response based on the majority consensus and give reason for choosing why that response
    """

        # Generate final response
        #finalResponse = Gmodel.generate_content(selfConsistencyPrompt, safety_settings={'HARASSMENT':'block_none'})
        finalResponse = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {
                "role":"system",
                "content":f"You are a Machine Learning engineer. Your task is to Pick the most consistent response based on the majority consensus and give reason for choosing why that response."
            },
               {
                "role": "user",
                "content": f"selfConsistencyPrompt:{selfConsistencyPrompt}\n\n.Pick the most consistent response based on the majority consensus and give reason for choosing why that response."
               }


        ]

    )
        # Extract response number from the text
        #Res = Gmodel.generate_content(f"Extract the response number from the text. Be very concise and output only and only the response number and nothing else {finalResponse.text}", safety_settings={'HARASSMENT':'block_none'})
        Res = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
               {
                "role": "user",
                "content": f"finalResponse:{finalResponse.choices[0].message.content}\n\n.Extract the response number from the finalResponse text. Be very concise and output only and only the response number and nothing else."
               }


        ]

    )
        
        Instruction=""
        if(InstructionModel=="gpt"):
          # Get instruction based on selected response
          Instruction = responses[int(Res.choices[0].message.content)]
        else:
          Instruction = responses[int(Res.text)]
        print(Instruction)  
        Pipeline=Instruction.choices[0].message.content     
        base_folder = expDetails['FolderName'].lstrip('/').replace('/', '\\')  # Convert forward slashes to backslashes
        expname = expDetails['ExpName'] + RunCount
        # Write instruction to pipeline.md in the created folder
        filepath = os.path.join(base_folder, expname, 'pipeline.md')
        os.makedirs(os.path.dirname(filepath), exist_ok=True)
        with open(filepath,'w') as f:
            print("here4")
            f.write(Pipeline)
        return True  
      except Exception as e:
        print("Error occurred:", e)
while True:
    if run_script():
        print("Script executed successfully.")
        break
    else:
        print("Error encountered. Retrying...")


ChatCompletion(id='chatcmpl-9gG04lLRKwjPVAz3y4WXMFEdGc4SC', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="The proposed methodology and its refinement offer a comprehensive plan for analyzing the performance of neural networks and gradient boosted decision trees on a tabular stock market dataset. The methodology correctly identifies key steps such as preprocessing, feature engineering, metafeature calculation, model training and evaluation, as well as metafeature analysis. This structured approach ensures a thorough comparison between different algorithms and provides insights into how dataset characteristics influence algorithm performance. Below are further elaborations and considerations for each step to ensure clarity and effectiveness in execution:\n\n1. **Preprocess the Dataset**:\n    - **Missing Values**: Implement strategies based on the nature of missing data. For example, median imputation for skewed distributions or mean

In [17]:
base_folder = expDetails['FolderName'].lstrip('/').replace('/', '\\')  # Convert forward slashes to backslashes
expname = expDetails['ExpName'] + RunCount
filepath = os.path.join(base_folder, expname, 'pipeline.md')
Pipeline=""
with open(filepath, 'r') as f:
  Pipeline=f.read()

Code generator

In [ ]:
train = pd.read_csv(custom_dataset)

In [21]:
completion = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {
                "role":"system",
                "content":f"You are a Machine Learning engineer. Your task is to generate very detailed & Complete Python code based on end to end.Don't ommit anything due to breveity , Don't miss out on any steps.Do all complicated implementations "
            },
               {
                "role": "user",
                "content": f"idea:{idea} \n\n pipeline:{Pipeline}\n\n\n miscellenous:{miscellaneous} \n\n..Assume the data is already loaded in train variable.Follow the instructions clearly.Do all the complicated implementations.Follow the [Instruction];{Pipeline} strictly."
            },
            {
                "role": "user",
                "content": f"Do all implementations without omission ignoring practical constraints.Don't simulate anything or don't include any Placeholders.Don't do brevity or use words like #pass"
            },
            {
                "role": "user",
                "content": f" [Idea]:{idea}\n\n Dataset_loading: Assume the data is already loaded in 'train'. The training data is stored in *train* variable.\n\n "
            }


        ],
        temperature=0.5

    )
print(completion.choices[0].message.content)

Based on the comprehensive methodology outlined above, let's now implement the Python code for each step in detail. We'll work through data preprocessing, feature engineering, model training, evaluation, and analysis without omitting any steps or using placeholders.

### Step 1: Preprocess the Data

```python
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

def preprocess_data(data):
    # Handle missing values
    imputer = SimpleImputer(strategy='mean')  # or median based on distribution
    data.iloc[:, :-1] = imputer.fit_transform(data.iloc[:, :-1])
    
    # Encode categorical features
    # Assuming categorical variables are of type 'object'
    categorical_cols = data.select_dtypes(include=['object']).columns
    encoder = OneHotEncoder(sparse=False)
    categorical_data = encoder.fit_transform(data[categorical_cols])
    categorical_df = pd.DataFram

In [23]:
import re
with open('completion[0].txt', 'w') as f:
    f.write(completion.choices[0].message.content)

# Print a confirmation message
print("Message content has been saved to 'completion[0].txt' file.")


# Print a confirmation message
print("Message content has been saved to 'completion[0].txt' file.")

# Read the content of the text file
with open('completion[0].txt', 'r') as file:
    text_content = file.read()

# Extract Python code blocks using regular expressions
python_code_blocks = re.findall(r'```python\s*([\s\S]+?)\s*```', text_content)

# Remove pip installs from Python code blocks
python_code_blocks_cleaned = []
for code_block in python_code_blocks:
    cleaned_block = re.sub(r'^!pip install .*\n', '', code_block, flags=re.MULTILINE)
    python_code_blocks_cleaned.append(cleaned_block)

# Write the cleaned Python code blocks to a new file
with open('extracted_code_cleaned.py', 'w') as file:
    for code_block in python_code_blocks_cleaned:
        file.write(code_block + '\n\n')


Message content has been saved to 'completion[0].txt' file.
Message content has been saved to 'completion[0].txt' file.


In [24]:
Code=completion.choices[0].message.content

In [49]:
import traceback
import re
def debug_error(error_message, code):
    # Define the prompt with the error message
    prompt = f"Error message: {error_message}\n\nDebugging suggestions:"

    # Generate suggestions using GPT
    completion = client.chat.completions.create(
    model="gpt-4-turbo-preview",
    system="You are a data scientist.I have encountered an error.Please fix the error in a logical way and dont assume anything.Regenerate the entire code again.If you need any clarifications ask.Output the entire code again after error rectification""",
    messages=[       {
                "role": "system",
                "content": "You are a data scientist.I have encountered an error.Please fix the error in a logical way and don't assume anything.Regenerate the entire code again.Output the entire rectified code again after error rectification",
            },
        {
                "role": "user",
                "content": f"[Error]:{prompt}\n\n [code]:{code}.Please fix the error.Don't comment out anything.Generate the complete code"
            },
                     {
                "role": "user",
                "content": f"pipeline:{Pipeline}\n\n [Context]:{context}\n\n [Idea]:{idea} [libraries]:{libraries}\n\n"
            }
                     ]

)

    # Extract and return the debugging suggestions
    suggestions = completion.choices[0].message.content
    print("Debugging suggestions:")
    print(suggestions)
    python_code_match = re.search(r'```python(.+?)```', suggestions, re.DOTALL)
    python_code = python_code_match.group(1) if python_code_match else ""

    # Check if the error is fixed based on the suggestions
    #new_error_message = suggestions.split("Error message:")[1].strip()
    return python_code



with open('extracted_code_cleaned.py', 'r') as f:
  python_code = f.read()
# Extract Python code from the provided message (example)
  python_code_blocks = re.findall(r'```python(.+?)```', python_code)
  print(python_code_blocks)
    # Prepare the code for execution
  code = """{}
         """.format(python_code_blocks)

  error_message = ""
  while True:
      try:
        # Execute the code
        print(code)
        exec(code)
        # If execution is successful, break the loop
        print("Code executed successfully. No errors.")
        base_folder = expDetails['FolderName'].lstrip('/').replace('/', '\\')
        expname = expDetails['ExpName'] + RunCount
        filepath = os.path.join(base_folder, expname, 'Basecode.py')
        # Ensure the directory exists
        os.makedirs(os.path.dirname(filepath), exist_ok=True)
        with open(filepath, 'w') as f:  # Changed 'r' to 'w' for writing
          f.write(code)
        print(f"Successfully wrote to {filepath}")
        expDetails['basecode_path'] = filepath

# Write the updated JSON back to expdetails.json
        with open('D:\LLM_GPT\APPCAIR\expDetails.json', 'w') as json_file:
          json.dump(expDetails, json_file, indent=2)
        break

      except (ValueError, Exception) as e:
            # Extract the error message
        error_message = str(e) + "\n" + traceback.format_exc()
        print("Error message:", error_message)

            # Debug the error message
        corrected_code = debug_error(error_message, code)
        print(error_message)

        if corrected_code:
          code = corrected_code
        else:
          print("Error correction failed. Exiting.")
          break


[]
[]
         
Code executed successfully. No errors.
Successfully wrote to D:\LLM_GPT\APPCAIR\TS_mixer17\Basecode.py


<>:68: SyntaxWarning: invalid escape sequence '\L'
<>:68: SyntaxWarning: invalid escape sequence '\L'
C:\Users\WIN 11\AppData\Local\Temp\ipykernel_25180\335155212.py:68: SyntaxWarning: invalid escape sequence '\L'
  with open('D:\LLM_GPT\APPCAIR\expDetails.json', 'w') as json_file:
